In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
import pandas as pd
import numpy as np
import plotnine as p9
import matplotlib.pyplot as plt
import ast

In [6]:
# date_time = "2023-04-08_17:38"
date_time = "2023-04-08_21:05"
date_time = "2023-04-09_15:31"
date_time = "2023-04-09_16:40"
date_time = "2023-06-01_14:32"
date_time = "2023-06-21_20:44" # necromass_bacteria

root_dir = "/projects/genomic-ml/da2343/ml_project_1/model_complexity/lasso_coef"
model_complexity_df = pd.read_csv(f"{root_dir}/lasso_coef_{date_time}.csv")

In [7]:
model_complexity_df
[-0.3700009927436331 -0.771553845352731 0.343301477202052 ...
 -2.0588130168127233e-05 4.638832947937765e-05 -1.1124252134971332e-06]

,subtrain_score,validation_score,reg_param,algorithm,data_set_name,fold_id,index_of_pred_col,coefs
0,3.189303e-13,0.789265,1.000000e-07,LASSO,necromass_bacteria,0,168,[-0.3700009927436331 -0.771553845352731 0.3433...
1,2.726366e-11,0.789312,1.000000e-06,LASSO,necromass_bacteria,0,168,[-0.3699995162749552 -0.7715562919524338 0.343...
2,1.566971e-09,0.788507,1.000000e-05,LASSO,necromass_bacteria,0,168,[-0.3700003095613319 -0.77157021442197 0.34330...
3,9.639279e-08,0.528307,1.000000e-04,LASSO,necromass_bacteria,0,168,[-0.17121202499985674 -0.5989497466723895 0.45...
4,2.636100e-06,0.015658,1.000000e-03,LASSO,necromass_bacteria,0,168,[-0.0 -0.0 0.525321083428301 ... -0.0 -0.0 0.0]
...,...,...,...,...,...,...,...,...
142267,9.129645e-01,1.184440,7.733344e-01,LASSO,necromass_bacteria,2,418,[0.0 0.0 -0.0 ... 0.0 -0.0 -0.0]
142268,1.017105e+00,1.247425,8.693280e-01,LASSO,necromass_bacteria,2,418,[0.0 0.0 -0.0 ... 0.0 -0.0 -0.0]
142269,1.059062e+00,1.265341,9.772372e-01,LASSO,necromass_bacteria,2,418,[0.0 0.0 -0.0 ... 0.0 -0.0 -0.0]
142270,1.059062e+00,1.265341,1.000000e+00,LASSO,necromass_bacteria,2,418,[0.0 0.0 -0.0 ... 0.0 -0.0 -0.0]


In [11]:
algorithm_list = model_complexity_df["algorithm"].unique()

for algorithm in algorithm_list:
    filtered_algorithm_df = model_complexity_df[model_complexity_df["algorithm"] == algorithm]
    dataset_list = filtered_algorithm_df["data_set_name"].unique().tolist()
    for dataset in dataset_list:
        # Get new dataframe with only the dataset
        sub_dataset_df = filtered_algorithm_df[filtered_algorithm_df["data_set_name"] == dataset]
        fold_id_list = sub_dataset_df["fold_id"].unique().tolist()
        
        for fold_id  in fold_id_list :
            test_error_df_list = []
            filtered_fold_id_df = sub_dataset_df[sub_dataset_df["fold_id"] == fold_id]
            reg_param_list = filtered_fold_id_df["reg_param"].unique().tolist()
            for reg_param in reg_param_list:
                filtered_reg_param = filtered_fold_id_df[filtered_fold_id_df["reg_param"] == reg_param]
                subtrain_score = filtered_reg_param['subtrain_score'].mean()
                validation_score = filtered_reg_param['validation_score'].mean()
                edges = 0
                index_of_pred_col_list = sorted(filtered_reg_param["index_of_pred_col"].unique().tolist())
                
                coef_matrix_list = []
                for index_of_pred_col in index_of_pred_col_list:
                    filtered_index_of_pred = filtered_reg_param[filtered_reg_param['index_of_pred_col'] == index_of_pred_col]
                    coefs = filtered_index_of_pred['coefs'].values.tolist()
                    coefs_str =  coefs[0]
                    # Replace "None" with "nan"
                    coefs_str = coefs_str.replace("None", "nan")
                    coefs_str = coefs_str.replace("\n", "")
                    coefs_str = coefs_str.replace("[", "")
                    coefs_str = coefs_str.replace("]", "")
                    
                    # Convert string to numpy array
                    coefs_arr = np.fromstring(coefs_str, sep=" ")
                    coef_matrix_list.append(coefs_arr)
                
                # Convert the list of numpy arrays into a matrix
                coefs_mat = np.array(coef_matrix_list)
                # get the indices of the upper and lower triangle elements
                upper_tri = np.triu_indices(coefs_mat.shape[0], k=1)
                lower_tri = np.tril_indices(coefs_mat.shape[0], k=-1)
                 # calculate the average of the upper and lower triangle elements
                avg_matrix = (coefs_mat + coefs_mat.T) / 2
                coefs_mat[lower_tri] = avg_matrix[lower_tri]
                coefs_mat[upper_tri] = np.nan
                np.fill_diagonal(coefs_mat, np.nan)
                
                # get the number of edges by counting the number of non-zero elements in the matrix
                # edges = np.count_nonzero(~np.isnan(coefs_mat))
                    # get the number of positive points in the array
                edges = np.count_nonzero(np.abs(coefs_mat)> 0)
                    # edges += local_edges
                # edges = edges/2
                # print(f"edges: {edges}")
                test_error_dict = {
                    'fold_id': fold_id,
                    'subtrain' :  subtrain_score,
                    'validation' : validation_score,
                    'data_set_name': dataset,
                    'reg_param': np.log10(reg_param),
                    'algorithm' : algorithm,
                    'edges': edges
                }
                # print(test_error_dict)
                test_error_df_list.append(pd.DataFrame(test_error_dict, index=[0]))
            test_err_df = pd.concat(test_error_df_list).reset_index()
            
            # Plot the results
            # make a plot of the mean train score and mean test score for each reg_param
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 5))
            test_err_df.plot(x="reg_param", 
                             y=['validation', 'subtrain'], 
                             ax=ax1,
                             #title=f'{algorithm} Model Complexity of {dataset} Dataset, FoldID:{fold_id}',
                             xlabel='', 
                             ylabel='Mean Squared Error', color = ['blue', 'red'], 
                             grid=True, 
                             legend=True,
                              )
            # ax1.invert_xaxis()
            ax1.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
            

            test_err_df.plot(x="reg_param", 
                             y= 'edges', 
                             ax=ax2,
                            #title=f'{algorithm} Model Complexity of {dataset} Dataset, FoldID:{fold_id}',
                            xlabel='log(λ)', 
                            ylabel='Number of Edges',
                            color = ['red'], 
                            legend=False,
                            grid=True
                        )
            # ax2.set_yticks([1176, 3000, 5000, 7000, 9000])
            
            # ax2.invert_xaxis()
            # mark only the best reg_param with a blue dot on both subplots
            best_reg_param = test_err_df.loc[test_err_df['validation'].idxmin()]['reg_param']
            best_validation = test_err_df.loc[test_err_df['validation'].idxmin()]['validation']
            best_edges = test_err_df.loc[test_err_df['validation'].idxmin()]['edges']

            ax1.scatter(best_reg_param, best_validation, color='blue')
            ax2.scatter(best_reg_param, best_edges, color='red')
            
            # Add a line to connect the two subplots
            line_x = [best_reg_param, best_reg_param]
            # # define the y-coordinates for the line
            line_y = np.array([np.min(test_err_df['subtrain']) - 0.01, np.max(test_err_df['validation'])])
            
            ax1.plot(line_x, line_y, 'k--')
            ax2.plot([best_reg_param, best_reg_param], np.array([np.min(test_err_df['edges']), np.max(test_err_df['edges'])]), 'k--')
            
            # draw a straight line between the two points
            line_x = [best_reg_param, best_reg_param]
            line_y = [best_validation, best_edges]
            plt.plot(line_x, line_y, color='black', linestyle='--')

            print(f"Best reg_param for {dataset} FoldID:{fold_id} is {best_reg_param}")
            print(f"Best no of edges for {dataset} FoldID:{fold_id} is {best_edges}")
            plt.subplots_adjust(hspace=0.0)
            
            # save the plot
            # plt.savefig(f"{algorithm}_{dataset}_foldid_{fold_id}_model_complexity.png", dpi=9000,  bbox_inches="tight")
            plt.show()
         
                   
            


/tmp/ipykernel_956143/225887718.py:34: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
